# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Get annotated tweets
epfl_df = pd.read_csv('../data/local/epfl_1000_clean_data.csv').reset_index()

epfl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         999 non-null    int64  
 1   id_tweets     998 non-null    float64
 2   text          999 non-null    object 
 3   id_doc        999 non-null    int64  
 4   sent_m        999 non-null    object 
 5   sent_c        999 non-null    object 
 6   sent_g        999 non-null    object 
 7   sent_l        620 non-null    object 
 8   neutral_sent  620 non-null    float64
 9   pos_sent      620 non-null    float64
 10  neg_sent      620 non-null    float64
 11  agree_sent    999 non-null    int64  
 12  neutral_stan  620 non-null    float64
 13  pos_stan      620 non-null    float64
 14  neg_stan      620 non-null    float64
 15  agree_stan    999 non-null    int64  
 16  stance_m      999 non-null    object 
 17  stance_c      999 non-null    object 
 18  stance_g      999 non-null    

# Prompts and loops

In [4]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user of the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [5]:
prompt_raw_1

'Give me the sentiment regarding vaccination expressed by the user of the tweet text delimited by triple backticks. Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. ```{TWEET_TEXT_HERE}```'

In [5]:
prompt_raw_0 = "Give me the sentiment regarding vaccination in tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. \
```{TWEET_TEXT_HERE}```"

In [6]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [7]:
prompt_raw_3 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral.\
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [8]:
prompt_raw_4 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [9]:
prompt_raw_5 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
* The user is ironic on people being negative towards vaccine (e.g., I am not getting a vaccine because they are microchipping you. Posted on social media , on a phone..that has all your info, your location...and the info is being shared and sold ...Ummmm ok) \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [10]:
prompt_raw_50 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Select a value between -2 (very negative), -1 (negative), 0 (neutral), +1 (positive) and +2 (very positive). Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as 0 except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as negative or very negative if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
* The user is ironic on people being negative towards vaccine (e.g., I am not getting a vaccine because they are microchipping you. Posted on social media , on a phone..that has all your info, your location...and the info is being shared and sold ...Ummmm ok) \
\
Examples of positive or very positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative or very negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [11]:
prompt_raw_6 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [12]:
prompt_raw_7 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
* Why are not more COVID-19 vaccine doses available immediately? <url> \
* America have fewer COVID-19 vaccines due to #MassMurderer #realDonaldTrumpâ€™s FAILED financial interests <url> \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
* Dr: Are you experiencing any side effects from the vaccine? Me: * sleeps while standing there* huh? \
* There is a virus which has killed so many people for years. This virus is called hunger and the vaccine for hunger is food. However no one talks about it, why? Because this virus doesn't kill rich people.\
* @user : There are over 100 different coronavirus vaccine candidates in the works. These candidates take a variety of approaches to protecting the body against COVID-19: <url> #CoronaVirusUpdates #CoronavirusVaccine #StayHome <url> \
* <It would go away without the vaccine> #HerdMentality this man knows nothing about viruses. \
* Anti-vaxxer Novak Djokovic. #COVID19 #tennis The current world number one is the fourth player to test positive from the event that did not even pay lip service to social distancing. <url> via @user \
\
```{TWEET_TEXT_HERE}```"

In [13]:
prompt_raw_8 = "Give me the stance of the user towards vaccination in the tweet text delimited by triple backticks, regardless of the general sentiment expressed in the tweet. A tweet with a negative general sentiment can express a positive stance of the user towards vaccination and viceversa. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the stance. \
The reply must contain the selected word for the stance followed by '.' and followed by the explanation for the selected stance. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
* The user is ironic on people being negative towards vaccine (e.g., I am not getting a vaccine because they are microchipping you. Posted on social media , on a phone..that has all your info, your location...and the info is being shared and sold ...Ummmm ok) \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [14]:
len(epfl_df.index)

999

In [ ]:
epfl_df.iloc[0,2]

In [28]:
id_gpt = []
sent_gpt = []
text = []

In [195]:
for i in range(0, 999):
    prompt_i = prompt_raw_5.replace('TWEET_TEXT_HERE', epfl_df.iloc[i,2])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    text.append(epfl_df.iloc[i,2])
    sent_gpt.append(response.choices[0].text)

In [39]:
for i in range(119, 999):
    prompt_i = prompt_raw_8.replace('TWEET_TEXT_HERE', epfl_df.iloc[i,2])
    response = openai.ChatCompletion.create(
      #model="text-davinci-003",
      model = "gpt-4",
      messages=[{"role": "user", "content": prompt_i}],
      temperature=0.8,
      #max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    text.append(epfl_df.iloc[i,2])
    sent_gpt.append(response.choices[0].message.content)

In [43]:
df_gpt_total = pd.DataFrame(list(zip(text, sent_gpt)),
                            columns = ['text', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           2277 non-null   object
 1   sentiment_gpt  2277 non-null   object
dtypes: object(2)
memory usage: 35.7+ KB


In [44]:
df_gpt_total['prompt'] = 48

In [45]:
df_gpt_total.tail()

,text,sentiment_gpt,prompt
2272,"@user , with these numbers, are you still goin...",Positive. The user is implying that proof of v...,48
2273,How the new Ebola vaccine was made in Canada <...,Neutral. The tweet is reporting a factual even...,48
2274,@user HOW TO END COVID-19 Give 3 free N95 mask...,Positive. The user is suggesting a comprehensi...,48
2275,Are you a bad person for not getting the vacci...,Neutral. The user is asking a question and doe...,48
2276,NICE TO KNOW: The JAPANESE ENCEPHALITIS VACCIN...,Neutral. The user is sharing a piece of inform...,48


In [46]:
len(df_gpt_total)

2277

In [47]:
df_gpt_total_all = df_gpt_total

In [ ]:
df_gpt_total_all = df_gpt_total_all.append(df_gpt_total)

In [48]:
len(df_gpt_total_all)

2277

In [49]:
df_gpt_total_all.tail()

,text,sentiment_gpt,prompt
2272,"@user , with these numbers, are you still goin...",Positive. The user is implying that proof of v...,48
2273,How the new Ebola vaccine was made in Canada <...,Neutral. The tweet is reporting a factual even...,48
2274,@user HOW TO END COVID-19 Give 3 free N95 mask...,Positive. The user is suggesting a comprehensi...,48
2275,Are you a bad person for not getting the vacci...,Neutral. The user is asking a question and doe...,48
2276,NICE TO KNOW: The JAPANESE ENCEPHALITIS VACCIN...,Neutral. The user is sharing a piece of inform...,48


In [50]:
df_gpt_total_all.to_csv('../data/local/gpt_sentiment_mturk_prompt48_new_new.csv')